In [ ]:
import numpy as np

# Import Pipeline class and datasets
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import USEquityPricing

from quantopian.pipeline.data.morningstar import Fundamentals

from quantopian.pipeline.data.factset import Fundamentals as factst_funda
from quantopian.pipeline.filters import Q500US, StaticSids, StaticAssets

# Import built-in trading universe
from quantopian.pipeline.filters import QTradableStocksUS


def make_pipeline():
    # Create a reference to our trading universe
    base_universe = QTradableStocksUS()

    # Get latest closing price
    close_price = USEquityPricing.close.latest
    
    #current_liabilities
    #The debts or obligations of the firm that are due within one year.
    current_liabilities = Fundamentals.current_liabilities.latest
    
    #total_assets
    #The aggregate amount of probable future economic benefits obtained or controlled by a particular enterprise as a result of past transactions or events.
    total_assets = Fundamentals.total_assets.latest
    market_cap = Fundamentals.market_cap.latest
    morningstar_industry_group_code = Fundamentals.morningstar_industry_group_code.latest
    morningstar_industry_code = Fundamentals.morningstar_industry_code.latest
    total_liabilities_net_minority_interest = Fundamentals.total_liabilities_net_minority_interest.latest
    operating_income = Fundamentals.operating_income.latest
    interest_expense = Fundamentals.interest_expense.latest
    cash_and_cash_equivalents = Fundamentals.cash_and_cash_equivalents.latest
    current_assets = Fundamentals.current_assets.latest
    current_liabilities = Fundamentals.current_liabilities.latest
    current_ratio = Fundamentals.current_ratio.latest
    debtto_assets = Fundamentals.debtto_assets.latest
    assets_of_discontinued_operations = Fundamentals.assets_of_discontinued_operations.latest
    # interest_expense
    long_term_debt = Fundamentals.long_term_debt.latest
    total_debt_equity_ratio = Fundamentals.total_debt_equity_ratio.latest
    
    # 貸倒れ引当金
    allowance_for_loans_and_lease_losses = Fundamentals.allowance_for_loans_and_lease_losses.latest
    
    myassets = StaticAssets(symbols(["PETS","SLP","USB"]))#'AAPL', 'FB', 'MSFT', 'AMZN', 'GOOG', 'KO'
    
    # Return Pipeline containing close_price and
    # sentiment_score that has our trading universe as screen
    return Pipeline(
        columns={
            'close_price': close_price,
#             'current_liabilities':current_liabilities, 
            'total_assets':total_assets, 
            'cash_and_cash_equivalents':cash_and_cash_equivalents,
            'current_assets':current_assets,
            'liab/asset':total_liabilities_net_minority_interest/total_assets,
            'cash/asset':cash_and_cash_equivalents/total_assets,
#             'market_cap':market_cap / 1000000000,
            'morningstar_industry_group_code':morningstar_industry_group_code,
            'morningstar_industry_code':morningstar_industry_code,
            'total_liabilities_net_minority_interest': total_liabilities_net_minority_interest, 
            'operating_income':operating_income, 
#             'interest_expense':interest_expense,
            'current_liabilities':current_liabilities, 
            'current_ratio':current_ratio,
            'debtto_assets':debtto_assets,
            'assets_of_discontinued_operations':assets_of_discontinued_operations, 
            'long_term_debt':long_term_debt, 
            'total_debt_equity_ratio':total_debt_equity_ratio, 
            'allowance_for_loans_and_lease_losses': allowance_for_loans_and_lease_losses, 
            
        },
        screen=base_universe #base_universe #myassets
    )

# Import run_pipeline method
from quantopian.research import run_pipeline

# Execute pipeline created by make_pipeline
# between start_date and end_date
pipeline_output = run_pipeline(
    make_pipeline(),
    start_date='2010-11-30',
    end_date='2020-4-25'
)

# Display last 10 rows
pipeline_output.tail(5)

## chapter 2



### No25 現金は最大の武器

+ （流動）資産の中に、どれくら現金及び現金同等物があるか
    1. 社債、株式の新規発行
    1. 事業、資産の売却
    1. 運転資金よりも多くの現金を稼ぎ出す
    
+ 大量の現金と有価証券を保有し、借入金がほとんどない会社
+ 過去７年のBSで確認

### Fundamentals 
+ [debtto_assets](https://www.quantopian.com/docs/data-reference/morningstar_fundamentals#debtto-assets)
    + 企業が総資産に対してどれだけの負債（長期負債と現在の負債の合計）をバランスシート上に持っているか
+ 'cash/asset' : cash_and_cash_equivalents/total_assets
+ [assets_of_discontinued_operations](https://www.quantopian.com/docs/data-reference/morningstar_fundamentals#assets-of-discontinued-operations)
    + 会社の事業の一部を処分したり、売却したりしたもの。


In [ ]:
df = pipeline_output.loc["2020-3-30"]
df = df[df["operating_income"] > 0]
df = df[["cash/asset", "debtto_assets",]].dropna()

df[df["debtto_assets"] < 0.003].sort_values(by=["cash/asset", "debtto_assets"], ascending=[False, True])

### 優良と思われる銘柄
+ [MPWR](https://finance.yahoo.com/quote/MPWR?p=MPWR&.tsrc=fin-srch)
- [MNST](https://finance.yahoo.com/quote/MNST/financials?p=MNST)
- [ISRG](https://finance.yahoo.com/quote/ISRG/financials?p=ISRG)



In [ ]:
candidates = pipeline_output.xs(symbols('EBS'), level=1)
bqend = candidates.resample("A-Dec").last()
bqend
candidates["operating_income"].plot()

## 38：長期借入金より短期借入金が多い銀行には投資してはいけない




In [ ]:
candidates = pipeline_output.xs(symbols('PETS'), level=1)
candidates

In [ ]:
region_bank = pipeline_output[pipeline_output["morningstar_industry_code"]==10320020]

In [ ]:
USB = region_bank.xs(symbols('USB'), level=1)


In [ ]:
morgage_bank["debtto_assets"].unstack().plot()